In [ ]:
from os import listdir
from os.path import isfile

In [ ]:
pasta = "requisicoes"    

In [ ]:
total_respondidas = 0
total_sem_responder  = 0
for item in listdir(pasta):
    caminho = f'{pasta}/{item}'
    caminho_resposta = f'{caminho}/resposta.json'
    if isfile(caminho_resposta):
        total_respondidas+=1
    if not isfile(caminho_resposta):
        total_sem_responder+=1


print("total: ", total_respondidas + total_sem_responder)
print("total respondidas: ",total_respondidas)
print("total sem responder",total_sem_responder)
